In [27]:
import os
import sys
# make sure your module sees these
DB_HOST = os.getenv("DB_HOST", "37.61.241.45")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "Khubeo_IA")
DB_USER = os.getenv("DB_USER", "postgres")
DB_PASS = os.getenv("DB_PASS", "Xjp2yCm$G36WR4E")



sys.path.append(os.getcwd())
print(os.getcwd())


/home/avalencia/ETL_vacantions_France_and_neighboors/srcs


#### Checks in which folder currently we are

In [28]:
# os module that is used to get the current working directory
import os
print("cwd =", os.getcwd())


cwd = /home/avalencia/ETL_vacantions_France_and_neighboors/srcs


In [29]:
# os.chdir () # change the current working directory to the specified path
os.chdir("/home/avalencia/ETL_vacantions_France_and_neighboors")  

In [30]:
from pathlib import Path
csv = Path("fr-en-calendrier-scolaire.csv").resolve()
print(csv, "exists:", csv.exists())


/home/avalencia/ETL_vacantions_France_and_neighboors/fr-en-calendrier-scolaire.csv exists: True


In [31]:

os.chdir("/home/avalencia/ETL_vacantions_France_and_neighboors/srcs")  
# now reload so that get_engine picks up the new env vars
from vacances_etl.t_vacances_etl import * 

# Vacacions Analysis


### 2. Verify DB connection

In [32]:
from sqlalchemy import text

# sanity‐check
engine = get_engine()
with engine.connect() as conn:
    print("✅ Connected:", conn.execute(text("SELECT 1")).scalar())

✅ Connected: 1


In [33]:
from vacances_etl.t_vacances_etl import load_csvs

test_df = load_csvs("../fr-en-calendrier-scolaire.csv")

In [34]:
test_df.head()

,Description,Population,Date de début,Date de fin,Académies,Zones,annee_scolaire
0,Vacances de la Toussaint,-,2017-10-21T00:00:00+02:00,2017-11-06T00:00:00+01:00,Limoges,Zone A,2017-2018
1,Vacances de la Toussaint,-,2017-10-21T00:00:00+02:00,2017-11-06T00:00:00+01:00,Poitiers,Zone A,2017-2018
2,Vacances de Noël,-,2017-12-23T00:00:00+01:00,2018-01-08T00:00:00+01:00,Limoges,Zone A,2017-2018
3,Vacances de Noël,-,2017-12-23T00:00:00+01:00,2018-01-08T00:00:00+01:00,Lyon,Zone A,2017-2018
4,Vacances d'Hiver,-,2018-02-10T00:00:00+01:00,2018-02-26T00:00:00+01:00,Clermont-Ferrand,Zone A,2017-2018


### 3. Run the ETL pipeline

In [35]:
from vacances_etl.t_vacances_etl import  run_etl


In [36]:

run_etl(
    [ "../fr-en-calendrier-scolaire.csv", "../fr-en-calendrier-scolaire-remaining.csv"]
)


✅ ETL complete: 2396 rows in t_vacances.


### 4. Valivc_date output table

In [37]:

import pandas as pd

# Read the table into a DataFrame
df = pd.read_sql_table('t_vacances_scolaires', engine)

In [38]:
df.columns
# Display the first few rows of the DataFrame


Index(['vac_date', 'vac_fr_zone_a', 'vac_fr_zone_b', 'vac_fr_zone_c',
       'vac_fr_corse', 'vac_all', 'vac_bel', 'vac_esp', 'vac_ita', 'vac_lux',
       'vac_sui', 'vac_annee_scolaire'],
      dtype='object')

In [39]:
df.head()  # Display the first few rows of the DataFrame

,vac_date,vac_fr_zone_a,vac_fr_zone_b,vac_fr_zone_c,vac_fr_corse,vac_all,vac_bel,vac_esp,vac_ita,vac_lux,vac_sui,vac_annee_scolaire
0,2009-10-26,0,0,0,1,0,0,0,0,0,0,2009-2010
1,2009-10-27,0,0,0,1,0,0,0,0,0,0,2009-2010
2,2009-10-28,0,0,0,1,0,0,0,0,0,0,2009-2010
3,2009-10-29,0,0,0,1,0,0,0,0,0,0,2009-2010
4,2009-10-30,0,0,0,1,0,0,0,0,0,0,2009-2010


### Check Summmer Vacances

In [41]:

# Filter the DataFrame for vac_dates between July 2, 2025 and September 3, 2025

summer_vacations_2025 = df[(df['vac_date'] >= '2025-07-02') & (df['vac_date'] <= '2025-09-03')]

# Set display options to show all rows and columns
with pd.option_context('display.max_rows', 20, 'display.max_columns', 20, 'display.width', None):
    # Display the resulting filtered DataFrame
    display(summer_vacations_2025)

# Print summary statistics
print(f"\nTotal summer vacation days: {len(summer_vacations_2025)}")
print(f"Start vac_date: {summer_vacations_2025['vac_date'].min()}")
print(f"End vac_date: {summer_vacations_2025['vac_date'].max()}")

# Show which zones have vacations during this period (count of days marked as 1)
zone_counts = summer_vacations_2025.iloc[:, 1:-1].sum()
print("\nNumber of vacation days by zone:")
print(zone_counts)

,vac_date,vac_fr_zone_a,vac_fr_zone_b,vac_fr_zone_c,vac_fr_corse,vac_all,vac_bel,vac_esp,vac_ita,vac_lux,vac_sui,vac_annee_scolaire
2111,2025-07-05,1,1,1,0,0,0,0,0,0,0,2024-2025
2112,2025-07-06,1,1,1,0,0,0,0,0,0,0,2024-2025
2113,2025-07-07,1,1,1,0,0,0,0,0,0,0,2024-2025
2114,2025-07-08,1,1,1,0,0,0,0,0,0,0,2024-2025
2115,2025-07-09,1,1,1,0,0,0,0,0,0,0,2024-2025
...,...,...,...,...,...,...,...,...,...,...,...,...
2167,2025-08-30,1,1,1,0,1,0,0,0,0,0,2024-2025
2168,2025-08-31,1,1,1,0,1,0,0,0,0,0,2024-2025
2169,2025-09-01,0,0,0,0,1,0,0,0,0,0,2025-2026
2170,2025-09-02,0,0,0,0,1,0,0,0,0,0,2025-2026



Total summer vacation days: 61
Start vac_date: 2025-07-05 00:00:00
End vac_date: 2025-09-03 00:00:00

Number of vacation days by zone:
vac_fr_zone_a    58
vac_fr_zone_b    58
vac_fr_zone_c    58
vac_fr_corse      0
vac_all          34
vac_bel           0
vac_esp           0
vac_ita           0
vac_lux           0
vac_sui           0
dtype: int64


### Check Winter Vacations

In [45]:

# 2. select the zone‑B winter break vac_dates
winter = df[(df["vac_date"] >= "2025-02-05") & (df["vac_date"] < "2025-02-27")]

# 3. check that all of those are flagged 1
# assert (winter["fr_zone_b"] == 1).all(), "Some winter days missing = 0?"
with pd.option_context('display.max_rows', 20, 'display.max_columns', 20, 'display.width', None):
    # Display the resulting filtered DataFrame
    display(winter)

# print("✅ Zone B winter break correctly mapped in t_vacances.")


,vac_date,vac_fr_zone_a,vac_fr_zone_b,vac_fr_zone_c,vac_fr_corse,vac_all,vac_bel,vac_esp,vac_ita,vac_lux,vac_sui,vac_annee_scolaire
2047,2025-02-08,0,1,0,0,0,0,0,0,0,0,2024-2025
2048,2025-02-09,0,1,0,0,0,0,0,0,0,0,2024-2025
2049,2025-02-10,0,1,0,0,0,0,0,0,0,0,2024-2025
2050,2025-02-11,0,1,0,0,0,0,0,0,0,0,2024-2025
2051,2025-02-12,0,1,0,0,0,0,0,0,0,0,2024-2025
2052,2025-02-13,0,1,0,0,0,0,0,0,0,0,2024-2025
2053,2025-02-14,0,1,0,0,0,0,0,0,0,0,2024-2025
2054,2025-02-15,0,1,1,1,0,0,0,0,0,0,2024-2025
2055,2025-02-16,0,1,1,1,0,0,0,0,0,0,2024-2025
2056,2025-02-17,0,1,1,1,0,0,0,0,0,0,2024-2025


## Vacances Periode 2025-2026

In [ ]:


# Ensure 'vac_date' column is datetime type
df['vac_date'] = pd.to_datetime(df['vac_date'])

# Filter for year 2025
df_2025 = df[df['vac_annee_scolaire'] == "2025-2026" ].copy() # Use .copy() to avoid SettingWithCopyWarning

# Select the French zone columns
# These are the name of the columsn in the DataFrame and in the database
zones = ['vac_all', 'vac_bel', 'vac_esp', 'vac_fr_corse', 'vac_fr_zone_a', 'vac_fr_zone_b',
       'vac_fr_zone_c', 'vac_ita', 'vac_lux', 'vac_sui']
# Ensure only existing columns are selected to avoid errors if a zone column is missing
existing_zones = [zone for zone in zones if zone in df_2025.columns]
df_zones_2025 = df_2025[existing_zones]

# Calculate the sum for each zone (this counts the number of days marked as 1, presumably holidays)
holiday_counts_2025 = df_zones_2025.sum()

# Display the resulting Series (index: zone name, values: count of holidays in 2025)
holiday_counts_2025

vac_all           57
vac_bel          103
vac_esp          103
vac_fr_corse       0
vac_fr_zone_a     68
vac_fr_zone_b     68
vac_fr_zone_c     68
vac_ita           28
vac_lux          118
vac_sui           71
dtype: int64